In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle

In [ ]:
data = pd.read_csv("Churn_Modelling.csv")

In [3]:
data = data.drop(columns=["RowNumber","CustomerId","Surname"])

In [4]:
label_encoder = LabelEncoder()
data["Gender"]= label_encoder.fit_transform(data["Gender"])

In [ ]:
one_hot_encoder = OneHotEncoder(drop='first')
geo_encoder = one_hot_encoder.fit_transform(data[["Geography"]])
geo_encoded_df = pd.DataFrame(geo_encoder, columns=one_hot_encoder.get_feature_names_out(["Geography"]))
data = pd.concat([data.drop(columns=["Geography"]), geo_encoded_df], axis=1)
data.drop(columns=["Geography"], axis=1, inplace=True)
data

ValueError: input_features is not equal to feature_names_in_